# Validate Forecast Skill vs. ERA5 or Observations

This example shows how to evaluate Salient's probabilistic forecasts against observations and calculate meaningful metrics. It demonstrates [validation best practices](https://salientpredictions.notion.site/Validation-0220c48b9460429fa86f577914ea5248) such as:

- Proper scoring using the Continuous Ranked Probability Score (CRPS)
  - Considers the full forecast distribution to reward both accuracy and precision
  - Less sensitive to climatology decisions than metrics like Anomaly Correlation
- A long backtesting period (2015-2022)
  - Short evaluation periods are subject to noise


In [ ]:
# Initialize the environment:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

sk.set_file_destination("validation_example")
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Customize The Validation

This notebook is written flexibly so you have the option of validating Salient and other forecasts multiple ways. These variables will control what, when, and how the validation proceeds.

The `split_set` variable controls the amount of data to request via the `start_date` and `end_date` variables.

- `sample` - a single season of data, good for quickly making sure that the mechanics of the process work.
- `test` - gets data from 2015-2022, which is completely out-of-sample from model training. This requests a medium amount of data and is recommended for most validation processes.
- `all` - gets data from 2000-2022, representing the full historical evaluation record. This will download quite a bit of data and is not recommended for most applications.


In [ ]:
# 1. Set the meteorological variable that we'll be evaluating:
var = "temp"  # temperature
# var = "precip"  # precipitation
# var = "wspd" # wind
# var = "tsi" # solar

# 2. Set the forecast look-ahead amount:
timescale = "sub-seasonal"  # weeks 1-5
# timescale = "seasonal"  # months 1-3
# timescale = "long-range" # quarters 1-4

# 3. Set the number of hindcasts to download for validation:
split_set = "sample"  # fast demonstration of mechanics
# split_set = "test"  # recommended to validate out-of-sample with hindcast_summary
# split_set = "all"  # download every hindcast since 2000

# 4. Set the reference model to compare Salient blend to
ref_model = "clim"  # Climatology.  Works across all timescale values.
# ref_model = "noaa_gefs"  # Valid for the sub-seasonal timescale
# ref_model = "ecmwf_ens" # Valid for sub-seasonal timescale
# ref_model = "ecmwf_seas5" # Valid for seasonal and long-range timescales

# 5. Use meteorological station observations in validation:
validate_obs = True  # Calculate skill of debiased forecast vs met stations
# validate_obs = False  # Calculate skill of undebiased forecast vs ERA5

# ===== Additional shared variables ==========================
fld = "vals"  # Evaluate in absolute space, not anomaly
model = "blend"  # Validate the primary Salient blend model
force = False  # If "False", cache data calls.  Set to "True" to overwrite caches

frequency = {"sub-seasonal": "weekly", "seasonal": "monthly", "long-range": "quarterly"}[timescale]
(start_date, end_date) = {
    "sample": ("2021-04-01", "2021-08-31"),
    "test": ("2015-01-01", "2022-12-31"),
    "all": ("2000-01-01", "2022-12-31"),
}[split_set]

## Set the Area of Interest

The Salient SDK uses a "Location" object to specify the geographic bounds of a request. In this case, we will be validating against the vector of airport locations that are used to settle the Chicago Mercantile Exchange's Cooling and Heating Degree Day contracts. With `load_location_file` we can see that the file contains:

- `lat` / `lon`: latitude and longitude of the met station, standard for a `location_file`
- `name`: the 3-letter IATA airport code of the location, also `location_file` standard
- `ghcnd`: the global climate network ID of the station, used to validate against observations. To customize this analysis for any set of observation stations, use the NCEI [stations list](https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt).
- `cme`: the CME code for the location used to create CDD/HDD strip codes.
- `description`: full name of the airport

If you have a list of locations already defined in a separate CSV file, you can use [`upload_file`](https://sdk.salientpredictions.com/api/#salientsdk.upload_file) to upload the file directly without building it in code via `upload_location_file`.


In [ ]:
# fmt: off
loc = sk.Location(location_file=sk.upload_location_file(
    lats =[33.62972     ,      42.36057,      34.19966,      41.96017,      39.04443,      32.89744,      29.98438,      36.07190,      44.88523,      40.77945,      39.87326,      45.59578,      38.50659],
    lons =[-84.44224    ,     -71.00975,    -118.36543,     -87.93164,     -84.67241,     -97.02196,     -95.36072,    -115.16343,     -93.23133,     -73.88027,     -75.22681,    -122.60919,    -121.49604],
    names=["ATL"        ,         "BOS",         "BUR",         "ORD",         "CVG",         "DFW",         "IAH",         "LAS",         "MSP",         "LGA",         "PHL",         "PDX",         "SAC"],
    ghcnd=["USW00013874", "USW00014739", "USW00023152", "USW00094846", "USW00093814", "USW00003927", "USW00012960", "USW00023169", "USW00014922", "USW00014732", "USW00013739", "USW00024229", "USW00023232"],
    cme  =["1"          ,           "W",           "P",           "2",           "3",           "5",           "R",           "0",           "Q",           "4",           "6",           "7",           "S"],
    geoname="cmeus",
    force=force,
    description=["Atlanta Hartsfield", "Boston Logan", "Burbank-Glendale-Pasadena", "Chicago O'Hare", "Cincinnati (Covington)","Dallas-Fort Worth", "Houston-George Bush", "Las Vegas McCarran", "Minneapolis-StPaul", "New York La Guardia","Philadelphia", "Portland", "Sacramento Exec"],
))
# fmt: on
stations = loc.load_location_file()
print(stations)

         lat        lon name        ghcnd cme                description                     geometry
0   33.62972  -84.44224  ATL  USW00013874   1         Atlanta Hartsfield   POINT (-84.44224 33.62972)
1   42.36057  -71.00975  BOS  USW00014739   W               Boston Logan   POINT (-71.00975 42.36057)
2   34.19966 -118.36543  BUR  USW00023152   P  Burbank-Glendale-Pasadena  POINT (-118.36543 34.19966)
3   41.96017  -87.93164  ORD  USW00094846   2             Chicago O'Hare   POINT (-87.93164 41.96017)
4   39.04443  -84.67241  CVG  USW00093814   3     Cincinnati (Covington)   POINT (-84.67241 39.04443)
5   32.89744  -97.02196  DFW  USW00003927   5          Dallas-Fort Worth   POINT (-97.02196 32.89744)
6   29.98438  -95.36072  IAH  USW00012960   R        Houston-George Bush   POINT (-95.36072 29.98438)
7   36.07190 -115.16343  LAS  USW00023169   0         Las Vegas McCarran   POINT (-115.16343 36.0719)
8   44.88523  -93.23133  MSP  USW00014922   Q         Minneapolis-StPaul   POINT (

## Pull precomputed skill

Salient pre-calculates skill metrics as part of our internal validation and model improvement process. Use the `hindcast_summary` api endpoint to request pre-calculated skill scores. This is the "easy" way to validate Salient's forecasts: we've already done all the work for you.

The remainder of this notebook will show you how to reproduce this skill calculation by requesting historical forecasts, historical data, and calculating a skill score.


In [ ]:
skill_summ = (
    pd.read_csv(
        sk.hindcast_summary(
            loc=loc,
            interp_method="linear",
            metric="crps",
            variable=var,
            timescale=timescale,
            reference=ref_model,
            split_set="test" if split_set == "sample" else split_set,
            verbose=False,
            force=force,
        )
    )
    .round(decimals=3)
    .drop(columns=["Reference Model"])
    .set_index(["Location", "Lead"])
    .sort_index(level=[0, 1])
)
print(skill_summ)

                 Reference CRPS (°C)  Salient CRPS (°C)  Salient CRPS Skill Score (%)
Location Lead                                                                        
ATL      Week 1                1.522              0.476                        68.705
         Week 2                1.521              1.063                        30.122
         Week 3                1.528              1.361                        10.951
         Week 4                1.525              1.410                         7.512
         Week 5                1.522              1.408                         7.504
...                              ...                ...                           ...
SAC      Week 1                1.218              0.487                        59.950
         Week 2                1.225              0.889                        27.396
         Week 3                1.225              1.087                        11.314
         Week 4                1.228              1.11

In [ ]:
def plot_skill(
    df: pd.DataFrame, col: str = "Salient CRPS Skill Score (%)", title: str = "Skill"
) -> None:
    """Plot skill scores in a table."""
    df = df.reset_index()

    plt.figure(figsize=(12, 6))
    for location in df["Location"].unique():
        subset = df[df["Location"] == location]
        plt.plot(subset["Lead"], subset[col], label=location)

    plt.title(title)
    plt.xlabel("Lead")
    plt.ylabel(col)
    plt.legend(title="Location", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()


plot_skill(skill_summ, title=f"hindcast_summary crps {model} vs {ref_model}")

## Historical Data

To calculate forecast skill, we will want to compare forecasts made in the past with actuals. There are two flavors of actual data: (1) The ERA5 reanalysis dataset and (2) point weather station observations.

Salient's forecast natively predicts (1) ERA5, but contains a debiasing function to remove bias between ERA5 and (2) station observations.


### Historical ERA5 Data

Download daily historical values from [`data_timeseries`](https://sdk.salientpredictions.com/api/#salientsdk.data_timeseries) and then aggregate to match the forecasts, so that we can ensure that all forecasts use the same dates.

Also, get observed weather station data in the same format by downloading directly from NCEI.


In [ ]:
# Get additional historical data beyond end_date to make sure we have enough
# observed days to compare with the final forecast.
duration = {"sub-seasonal": 8 * 5, "seasonal": 31 * 3, "long-range": 95 * 4}[timescale]
hist = sk.data_timeseries(
    loc=loc,
    variable=var,
    field=fld,
    start=np.datetime64(start_date) - np.timedelta64(5, "D"),
    end=np.datetime64(end_date) + np.timedelta64(duration, "D"),
    frequency="daily",
    # reference_clim="30_yr",  implicitly uses 30 yr climatology
    verbose=False,
    force=force,
)
print(xr.load_dataset(hist))

<xarray.Dataset> Size: 23kB
Dimensions:   (time: 198, location: 13)
Coordinates:
  * time      (time) datetime64[ns] 2kB 2021-03-27 2021-03-28 ... 2021-10-10
    lat       (location) float64 104B 33.63 42.36 34.2 ... 39.87 45.6 38.51
    lon       (location) float64 104B -84.44 -71.01 -118.4 ... -122.6 -121.5
  * location  (location) <U3 156B 'ATL' 'BOS' 'BUR' 'ORD' ... 'PHL' 'PDX' 'SAC'
Data variables:
    vals      (time, location) float64 21kB 21.77 10.66 14.5 ... 12.31 16.97
Attributes:
    long_name:   2 metre temperature
    units:       degC
    clim_start:  1990-01-01
    clim_end:    2019-12-31


## Forecast

The [`forecast_timeseries`](https://sdk.salientpredictions.com/api/#salientsdk.forecast_timeseries) API endpoint and SDK function returns Salient's native temporally granular weekly/monthly/quarterly forecasts.

This is the most heavyweight call in the notebook, since it's getting multiple historical forecasts.


In [ ]:
# get_hindcast_dates is a utility that returns all valid hindcast initializations.
date_range = sk.get_hindcast_dates(start_date=start_date, end_date=end_date, timescale=timescale)

fcst = sk.forecast_timeseries(
    loc=loc,
    variable=var,
    field=fld,
    date=date_range,
    timescale=timescale,
    model=[model, ref_model],
    reference_clim="30_yr",  # this is the same climatology used by data_timeseries
    debias=False,
    verbose=False,
    force=force,
    strict=False,  # There is missing data in 2020.  Work around it.
)
print(fcst)

                                             file_name        date  model
0    validation_example/forecast_timeseries_2080b77...  2021-04-01  blend
1    validation_example/forecast_timeseries_d2f085b...  2021-04-01   clim
2    validation_example/forecast_timeseries_3e8ae2d...  2021-04-04  blend
3    validation_example/forecast_timeseries_cbd0d20...  2021-04-04   clim
4    validation_example/forecast_timeseries_f8f2b29...  2021-04-07  blend
..                                                 ...         ...    ...
129  validation_example/forecast_timeseries_c8001f4...  2021-08-25   clim
130  validation_example/forecast_timeseries_cce5434...  2021-08-28  blend
131  validation_example/forecast_timeseries_b1e01ed...  2021-08-28   clim
132  validation_example/forecast_timeseries_dcbf551...  2021-08-31  blend
133  validation_example/forecast_timeseries_0547de0...  2021-08-31   clim

[134 rows x 3 columns]


In [ ]:
# Check to see if there are any missing forecasts:
fcst_na = fcst[fcst["file_name"].isna()]
if not fcst_na.empty:
    print("Missing forecast dates:")
    print(fcst_na)

# Example forecast file is for a single model and a single forecast_date
print(xr.load_dataset(fcst["file_name"].values[0]))

<xarray.Dataset> Size: 7kB
Dimensions:                 (lead_weekly: 5, location: 13, quantiles: 23,
                             nbnds: 2)
Coordinates:
    forecast_period_weekly  (lead_weekly, nbnds) datetime64[ns] 80B 2021-04-0...
    forecast_date_weekly    datetime64[ns] 8B 2021-04-01
  * lead_weekly             (lead_weekly) int64 40B 1 2 3 4 5
  * quantiles               (quantiles) float64 184B 0.01 0.025 ... 0.975 0.99
    lat                     (location) float64 104B 33.63 42.36 ... 45.6 38.51
    lon                     (location) float64 104B -84.44 -71.01 ... -121.5
  * location                (location) <U3 156B 'ATL' 'BOS' ... 'PDX' 'SAC'
Dimensions without coordinates: nbnds
Data variables:
    vals_weekly             (lead_weekly, location, quantiles) float32 6kB 9....
Attributes:
    clim_period:  ['1990-01-01', '2019-12-31']
    region:       north-america
    short_name:   temp
    timescale:    sub-seasonal


## Calculate Skill Metrics

Compare the forecast and ERA5 datasets to see how well they match. Here we will calculate the same "Continuous Ranked Probability Score" that resulted from the call to `hindcast_summary` earlier.


In [ ]:
skill_model = sk.skill.crps(
    observations=hist,
    forecasts=fcst[fcst["model"] == model],
)
print(skill_model)

<xarray.Dataset> Size: 42kB
Dimensions:                 (forecast_date_weekly: 67, lead_weekly: 5,
                             location: 13, nbnds: 2)
Coordinates:
    lat                     (location) float64 104B 33.63 42.36 ... 45.6 38.51
    lon                     (location) float64 104B -84.44 -71.01 ... -121.5
  * location                (location) <U3 156B 'ATL' 'BOS' ... 'PDX' 'SAC'
  * lead_weekly             (lead_weekly) int64 40B 1 2 3 4 5
  * forecast_date_weekly    (forecast_date_weekly) datetime64[ns] 536B 2021-0...
    forecast_period_weekly  (forecast_date_weekly, lead_weekly, nbnds) datetime64[ns] 5kB ...
Dimensions without coordinates: nbnds
Data variables:
    crps_weekly_all         (forecast_date_weekly, lead_weekly, location) float64 35kB ...
    crps_weekly             (lead_weekly, location) float64 520B 0.3575 ... 0...


The `crps_weekly` value shows the average of all error values over the full date range. `crps_weekly_all` preserves the crps for each `forecast_date`, so we can also show that CRPS varies over time and has a seasonal component:


In [ ]:
skill_model[f"crps_{frequency}_all"].mean(dim=f"lead_{frequency}").rolling(
    **{
        f"forecast_date_{frequency}": max(
            1, int(len(skill_model[f"forecast_date_{frequency}"]) * 0.1)
        )
    },
    center=True,
).mean().plot(x=f"forecast_date_{frequency}", hue="location", figsize=(12, 6))

units = xr.load_dataset(hist).attrs["units"]
plt.title("Error varies over time")
plt.ylabel(f"{model} crps {var} [{units}]")
plt.xlabel("");

Also calculate the skill of the "reference" model for purposes of comparison:


In [ ]:
skill_ref = sk.skill.crps(
    observations=hist,
    forecasts=fcst[fcst["model"] == ref_model],
)
print(skill_ref)

<xarray.Dataset> Size: 42kB
Dimensions:                 (forecast_date_weekly: 67, lead_weekly: 5,
                             location: 13, nbnds: 2)
Coordinates:
    lat                     (location) float64 104B 33.63 42.36 ... 45.6 38.51
    lon                     (location) float64 104B -84.44 -71.01 ... -121.5
  * location                (location) <U3 156B 'ATL' 'BOS' ... 'PDX' 'SAC'
  * lead_weekly             (lead_weekly) int64 40B 1 2 3 4 5
  * forecast_date_weekly    (forecast_date_weekly) datetime64[ns] 536B 2021-0...
    forecast_period_weekly  (forecast_date_weekly, lead_weekly, nbnds) datetime64[ns] 5kB ...
Dimensions without coordinates: nbnds
Data variables:
    crps_weekly_all         (forecast_date_weekly, lead_weekly, location) float64 35kB ...
    crps_weekly             (lead_weekly, location) float64 520B 0.787 ... 1.147


In [ ]:
skills = xr.concat(
    [
        skill_ref.assign_coords(source=ref_model),
        skill_model.assign_coords(source=model),
    ],
    dim="source",
)
print(skills)

<xarray.Dataset> Size: 77kB
Dimensions:                 (source: 2, forecast_date_weekly: 67,
                             lead_weekly: 5, location: 13, nbnds: 2)
Coordinates:
    lat                     (location) float64 104B 33.63 42.36 ... 45.6 38.51
    lon                     (location) float64 104B -84.44 -71.01 ... -121.5
  * location                (location) <U3 156B 'ATL' 'BOS' ... 'PDX' 'SAC'
  * lead_weekly             (lead_weekly) int64 40B 1 2 3 4 5
  * forecast_date_weekly    (forecast_date_weekly) datetime64[ns] 536B 2021-0...
    forecast_period_weekly  (forecast_date_weekly, lead_weekly, nbnds) datetime64[ns] 5kB ...
  * source                  (source) <U5 40B 'clim' 'blend'
Dimensions without coordinates: nbnds
Data variables:
    crps_weekly_all         (source, forecast_date_weekly, lead_weekly, location) float64 70kB ...
    crps_weekly             (source, lead_weekly, location) float64 1kB 0.787...


In [ ]:
def compare_model_skill(skills: xr.Dataset):
    """Show CRPS by lead, for 2 or more models."""
    skills[f"crps_{frequency}"].plot.line(
        x=f"lead_{frequency}", hue="source", col="location", col_wrap=3, figsize=(10, 10)
    )
    plt.suptitle(f"{var} CRPS", fontsize=16)
    plt.subplots_adjust(top=0.9)
    plt.show()


compare_model_skill(skills)

### Calculate Relative Skill

CRPS shows skill without context. A "skill score" will compare two different skills to generate relative value. In the example below, we will compare the Salient blend with climatology (historical averages).


In [ ]:
skill_score = sk.skill.crpss(forecast=skill_model, reference=skill_ref)

# Represent the skill scores as a human-readable table of the same format as we generated
# for the hindcast_summary results.
skill_table = (
    xr.merge(
        [
            (skill_ref[f"crps_{frequency}"].rename("Reference CRPS")).round(2),
            skill_model[f"crps_{frequency}"].rename("Salient CRPS").round(2),
            (skill_score[f"crpss_{frequency}"] * 100)
            .rename("Salient CRPS Skill Score (%)")
            .round(1),
        ]
    )
    .to_dataframe()
    .reset_index()
    .dropna(how="any")
    .drop(columns=["lat", "lon"])
    .rename(columns={"location": "Location", f"lead_{frequency}": "Lead"})
)
skill_table["Lead"] = "Week " + skill_table["Lead"].astype(str)
skill_table.set_index(["Location", "Lead"], inplace=True)

print(skill_table)

                 Reference CRPS  Salient CRPS  Salient CRPS Skill Score (%)
Location Lead                                                              
ATL      Week 1            0.79          0.36                          54.6
         Week 2            0.73          0.69                           6.2
         Week 3            0.69          0.75                          -8.2
         Week 4            0.66          0.73                          -9.4
         Week 5            0.66          0.77                         -15.7
...                         ...           ...                           ...
SAC      Week 1            1.13          0.44                          60.7
         Week 2            1.19          0.91                          23.8
         Week 3            1.18          1.03                          12.6
         Week 4            1.17          0.97                          17.1
         Week 5            1.15          0.97                          15.8

[65 rows x 

In [ ]:
plot_skill(skill_summ, title=f"manually-calculated crps {model} vs {ref_model}")

### Compare manually-calculated to pre-computed skill

Now that we have a CRPS calculated manually as well as downloaded from `hindcast_summary` we can evaluate how close the two values are.


In [ ]:
skill_merge = pd.merge(
    skill_summ.add_prefix("Summary "),
    skill_table.add_prefix("Manual "),
    left_index=True,
    right_index=True,
)

print(skill_merge)

Now let's visualize the manually-calculated skill score with the precomputed skill scores published by `hindcast_summary`.

Note that when using `split_set = sample` the values won't match exactly. In this case we are plotting skill scores calculated from a single year of forecasts against the skill scores from the `test` set.


In [ ]:
def compare_cols(col_name: str) -> None:
    """Plot manual and precalculated skill columns."""
    summary_col = f"Summary {col_name}"
    manual_col = f"Manual {col_name}"

    df = skill_merge.reset_index()

    plt.figure(figsize=(10, 6))

    for location in df["Location"].unique():
        subset = df[df["Location"] == location]
        plt.scatter(subset[summary_col], subset[manual_col], label=location, s=100)

    # Same limits for both axes
    min_limit = min(df[summary_col].min(), df[manual_col].min())
    max_limit = max(df[summary_col].max(), df[manual_col].max())
    plt.xlim(min_limit, max_limit)
    plt.ylim(min_limit, max_limit)
    plt.plot(
        [min_limit, max_limit], [min_limit, max_limit], color="gray", linestyle="--", linewidth=1
    )
    plt.gca().set_aspect("equal", adjustable="box")

    plt.title(f"Summary vs Manual {col_name}")
    plt.xlabel(summary_col)
    plt.ylabel(manual_col)
    plt.legend(title="Location", bbox_to_anchor=(1.05, 1), loc="upper left")

    if split_set == "sample":
        plt.text(
            min_limit,
            max_limit,
            "Results not expected to match for split_set = 'summary'.\nUse split_set = 'test' for a full comparison.",
            fontsize=10,
            verticalalignment="top",
            horizontalalignment="left",
            color="red",
        )

    plt.tight_layout()

    # Show the plot
    plt.show()


compare_cols("Salient CRPS Skill Score (%)")
# compare_cols("Salient CRPS")
# compare_cols("Reference CRPS")

## Validating vs Observation Station Data

Get observed historical data from meteorological stations. In this case, we'll write a `get_ghcnd` function that downloads observed station data and returns a list of pandas `DataFrame`s. If you have observed data from a proprietary source, you can substitute a function here that returns a vector of `DataFrame`s. Make sure that the `DataFrame`s have a column that matches the `variable` of interest (such as `temp`).


In [ ]:
if validate_obs:
    duration = {"sub-seasonal": 8 * 5, "seasonal": 31 * 3, "long-range": 95 * 4}[timescale]
    obs_src = sk.met_observations(
        loc=loc,
        variables=var,
        start=np.datetime64(start_date) - np.timedelta64(5, "D"),
        end=np.datetime64(end_date) + np.timedelta64(duration, "D"),
        verbose=False,
        force=force,
    )
    # Reformat observations to match the output of data_timeseries:
    obs = xr.load_dataset(obs_src).rename({var: "vals"})
    obs = obs.assign_attrs(**obs.vals.attrs, short_name=var)

    # Make sure that we found the expected set of stations:
    assert obs.station.values.tolist() == stations.ghcnd.to_list()
    # Lat-lon differences may complicate merging later:
    obs = obs.reset_coords(drop=True)
    print(obs)
else:
    print("skipped: not comparing to observed data")
    obs = None

<xarray.Dataset> Size: 12kB
Dimensions:   (time: 198, location: 13)
Coordinates:
  * time      (time) datetime64[ns] 2kB 2021-03-27 2021-03-28 ... 2021-10-10
  * location  (location) <U3 156B 'ATL' 'BOS' 'BUR' 'ORD' ... 'PHL' 'PDX' 'SAC'
Data variables:
    vals      (time, location) float32 10kB 22.8 10.55 16.1 ... 11.65 16.95
Attributes:
    long_name:   Average temperature at 2m
    units:       degC
    short_name:  temp


#### Compare observed and ERA5 datasets

Via `make_observed_ds`, the observed station data (`obs`) is formatted the same as the ERA5 historical data (`hist`). This means we can easily compare one to the other and see the degree of bias that exists between the two.


In [ ]:
if validate_obs:
    # Pull observed and ERA5 into a single dataset for easy comparison:
    merged = xr.merge(
        [
            obs.rename({"vals": "obs"}),
            xr.load_dataset(hist).rename({"vals": "hist"}),
        ],
        join="inner",
    )
    merged["delta_raw"] = merged["obs"] - merged["hist"]
    # Daily bias is noisy.  Smooth it out to make trends clearer.
    # This will induce nans at the beginning and end of the timeseries.
    merged["delta"] = (
        merged["delta_raw"]
        .rolling(time=max(1, int(len(merged["time"]) * 0.1)), center=True)
        .mean()
    )

    print(merged)
else:
    print("skipped: not comparing to observed data")
    merged = None

<xarray.Dataset> Size: 74kB
Dimensions:    (time: 198, location: 13)
Coordinates:
  * time       (time) datetime64[ns] 2kB 2021-03-27 2021-03-28 ... 2021-10-10
  * location   (location) <U3 156B 'ATL' 'BOS' 'BUR' 'ORD' ... 'PHL' 'PDX' 'SAC'
    lat        (location) float64 104B 33.63 42.36 34.2 ... 39.87 45.6 38.51
    lon        (location) float64 104B -84.44 -71.01 -118.4 ... -122.6 -121.5
Data variables:
    obs        (time, location) float32 10kB 22.8 10.55 16.1 ... 11.65 16.95
    hist       (time, location) float64 21kB 21.77 10.66 14.5 ... 12.31 16.97
    delta_raw  (time, location) float64 21kB 1.03 -0.1126 ... -0.662 -0.02339
    delta      (time, location) float64 21kB nan nan nan nan ... nan nan nan nan
Attributes:
    long_name:   Average temperature at 2m
    units:       degC
    short_name:  temp


In [ ]:
if validate_obs:
    # Visualize obs-era5 bias over time at each station
    merged["delta"].plot.line(x="time", hue="location")
    plt.axhline(y=0, color="k", linestyle="--", alpha=0.5)
    plt.title("Delta of Observed to Historical Values")
    plt.ylabel(f'{merged.attrs["long_name"]} obs - hist [{merged.attrs["units"]}]')
    plt.xlabel("")
    plt.grid(True, alpha=0.3)
    plt.show()

### Demonstrate the need for station debiasing

Salient's forecasts are trained to predict the ERA5 reanalysis. The bias between station observations and ERA5 means that native forecast skill will be lower than desired when compared to station observations. Let's calculate that undebiased skill so we can quantify the magnitude of the effect.


In [ ]:
if validate_obs:
    # Explicitly flag the native forecasts as undebiased:
    fcst["debias"] = False
    skill_undebiased_obs = sk.skill.crps(
        observations=obs,  # note that we're using observation stations as "truth"
        forecasts=fcst[(fcst["model"] == model) & (fcst["debias"] == False)],
    ).round(2)

In some cases where there is not much bias between ERA5 and the observation station (Philadelphia, Cincinnati) we see that there is also not much difference in skill depending on the source of truth. In other locations like Boston or New York (LGA) we see that the difference between the observation station and ERA5 induces nontrivial skill degradation.


In [ ]:
if validate_obs:
    skills_obs = xr.concat(
        [
            skill_undebiased_obs.assign_coords(source="undebiased vs obs"),
            skill_model.assign_coords(source="undebiased vs ERA5"),
        ],
        dim="source",
        coords="minimal",
    )

    compare_model_skill(skills_obs)

### Get debiased forecasts

Salient's models natively predict ERA5 values, not observation station values. The `forecast_timeseries`, `data_timeseries`, and `downscale` functions have a `debias` option that adjusts ERA5-based data to more closely match obserations. Let's get a set of forecasts with debiasing on.


In [ ]:
# Get debiased hindcasts, if we are validating vs observations
if validate_obs:
    fcst_debias = sk.forecast_timeseries(
        loc=loc,
        variable=var,
        field=fld,
        date=date_range,
        timescale=timescale,
        model=model,  # debias only avilable for model blend
        reference_clim="30_yr",
        debias=True,  # Note debias
        verbose=False,
        force=force,
        strict=False,
    )
    # Add synthetic columns so that the two forecast tables can concatenate:
    fcst_debias["model"] = model
    fcst_debias["debias"] = True

    fcst = pd.concat([fcst, fcst_debias], axis=0)

### Compare undebiased forecast skill to debiased

Now quantify the effect of debiasing by calculating the skill of the debiased forecast against observation station data.


In [ ]:
if validate_obs:
    skill_debiased_obs = sk.skill.crps(
        observations=obs,
        forecasts=fcst[(fcst["model"] == model) & (fcst["debias"] == True)],
    ).round(2)

The goal of debiasing is to bring the debiased-obs forecast skill (green) as close as possible to the undebiased-era5 skill (orange).

- Atlanta, Portland, Minneapolis, Philadelphia, and Sacramento have consistent offsets that result in near-perfect debiasing.
- Boston, Burbank, and New York (LGA) see a significant reduction in error as a result of debiasing, but there is still some residual skill loss.
- Chicago (ORD)'s observation-ERA5 offset is inconsistent, so the debiaser recovers limited skill.
- Cincinnati (CVG) had little bias to begin with, so the debiaser doesn't change the results.


In [ ]:
if validate_obs:
    skills_obs = xr.concat(
        [
            skill_undebiased_obs.assign_coords(source="undebiased vs obs"),
            skill_model.assign_coords(source="undebiased vs ERA5"),
            skill_debiased_obs.assign_coords(source="debiased vs obs"),
        ],
        dim="source",
        coords="minimal",
    )

    compare_model_skill(skills_obs)

Now we can attribute the incremental error of debiasing against observations. The relatively small size of the orange bars indicates that debiased (vs obs) forecasts are only slightly less skillful than the native (vs era5) forecasts.


In [ ]:
combined_skills = xr.concat(
    [
        skill_model.assign_coords(source="era5 forecast"),
        (skill_debiased_obs - skill_model).assign_coords(source="obs-era5"),
    ],
    dim="source",
    coords="minimal",
).mean(dim=f"lead_{frequency}")[f"crps_{frequency}"]

location_order = (
    skill_debiased_obs.mean(dim=f"lead_{frequency}")
    .sortby(f"crps_{frequency}", ascending=False)
    .location
)

combined_skills.reindex(location=location_order).T.to_pandas().plot(
    kind="bar", stacked=True, figsize=(12, 6)
)

plt.ylabel(f"CRPS {var} (all leads)")
plt.title("Error Contribution by Source");